In [19]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import gmaps
import time
from us import states
import gmaps.datasets

# Google developer API key
from config import gkey

In [20]:
# Reference the file where the CSV is located
population_csv_path = "../Project1/Population_by_Zip.csv"

In [21]:
# Import the data into a Pandas DataFrame
pop_by_zip_df = pd.read_csv(population_csv_path)
pop_by_zip_df.head()

,Zip Code ZCTA,2010 Census Population
0,1001,16769
1,1002,29049
2,1003,10372
3,1005,5079
4,1007,14649


In [22]:
# Same number of values in both rows
# Both columns are made up of integers
pop_by_zip_df.count()

Zip Code ZCTA             33092
2010 Census Population    33092
dtype: int64

In [23]:
# Chicago Zipcodes range from 60007 - 60827

In [24]:
chicago = pop_by_zip_df.loc[pop_by_zip_df['Zip Code ZCTA'] == 60603]
chicago

,Zip Code ZCTA,2010 Census Population
20615,60603,493


In [25]:
chicago_zip = pop_by_zip_df.loc[(pop_by_zip_df['Zip Code ZCTA'] >= 60007) & (pop_by_zip_df['Zip Code ZCTA'] <= 60827)]
# chicago_zip

In [26]:
# Change column titles 
chicago_pop_df = chicago_zip.rename(columns={"Zip Code ZCTA":"Chicago Zip Code", 
                                               "2010 Census Population":"Population in 2010"})
# chicago_pop_df.head(20)

In [27]:
chicago_pop_df.reset_index(drop=True)

,Chicago Zip Code,Population in 2010
0,60007,33820
1,60008,22717
2,60010,44095
3,60012,11120
4,60013,26872
5,60014,48550
6,60015,26800
7,60016,59690
8,60018,30099
9,60020,9825


In [28]:
chicago_pop_df.count()

Chicago Zip Code      337
Population in 2010    337
dtype: int64

In [29]:
chicago_pop_df

,Chicago Zip Code,Population in 2010
20340,60007,33820
20341,60008,22717
20342,60010,44095
20343,60012,11120
20344,60013,26872
20345,60014,48550
20346,60015,26800
20347,60016,59690
20348,60018,30099
20349,60020,9825


In [32]:
lat_long_csv = "../Project1/KB_US_Zip_Codes.csv"

In [33]:
zip_lat_long_df = pd.read_csv(lat_long_csv)
zip_lat_long_df.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135
5,612,18.402253,-66.711397
6,616,18.420412,-66.671979
7,617,18.445147,-66.559696
8,622,17.991245,-67.153993
9,623,18.083361,-67.153897


Heat Map - Zipcode and Population

In [25]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [29]:
fig = gmaps.figure(map_type='SATELLITE')

In [30]:
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
locations = chicago_pop_df["Chicago Zip Code"]

In [27]:
population = chicago_pop_df["Population in 2010"]

In [28]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(locations, weights=population, dissipating=False, max_intensity=100, point_radius=1)

# heat_layer.dissipating = False
# heat_layer.max_intensity = 100
# heat_layer.point_radius = 1

# fig.add_layer(heat_layer)

# fig

TypeError: 'int' object is not iterable